In [10]:
"""Cell 1: Setup and Imports"""
import sys
import os
import time
import warnings
import numpy as np
from IPython.display import clear_output

warnings.filterwarnings('ignore')

# Ensure src is in path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Clear all qectostim module cache BEFORE importing (fresh start)
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

# Also clear the bytecode cache
import importlib
importlib.invalidate_caches()

# Import testing utilities from qectostim.testing
from qectostim.testing import (
    clear_qectostim_modules,
    test_composite_construction,
    test_code_circuit,
    test_decoder_on_code,
    load_all_decoders,
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
)

# Import qectostim
from qectostim.codes import discover_all_codes
from qectostim.codes.composite import (
    ConcatenatedTopologicalCSSCode,
    HomologicalProductCode,
    DualCode,
)
from qectostim.codes.abstract_css import CSSCodeWithComplex
from qectostim.decoders import PyMatchingDecoder

# Load all available decoders
decoder_classes = load_all_decoders()

# Try to import ConcatenatedDecoder
try:
    from qectostim.decoders.concatenated_decoder import ConcatenatedDecoder
    HAS_CONCAT_DECODER = True
except ImportError as e:
    HAS_CONCAT_DECODER = False
    print(f"Note: ConcatenatedDecoder not available: {e}")

print("✓ All imports successful")
print(f"  ConcatenatedTopologicalCSSCode: {ConcatenatedTopologicalCSSCode}")
print(f"  HomologicalProductCode: {HomologicalProductCode}")
print(f"  DualCode: {DualCode}")
print(f"  Decoders loaded: {list(decoder_classes.keys())}")
print(f"  ConcatenatedDecoder available: {HAS_CONCAT_DECODER}")

✓ All imports successful
  ConcatenatedTopologicalCSSCode: <class 'qectostim.codes.composite.concatenated.ConcatenatedTopologicalCSSCode'>
  HomologicalProductCode: <class 'qectostim.codes.composite.homological_product.HomologicalProductCode'>
  DualCode: <class 'qectostim.codes.composite.dual.DualCode'>
  Decoders loaded: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat']
  ConcatenatedDecoder available: True


In [11]:
"""Cell 2: Discover Building Block Codes"""
clear_output(wait=True)

print("="*70)
print("DISCOVERING BUILDING BLOCK CODES")
print("="*70)

# Discover codes - use smaller size for composite building
all_codes = discover_all_codes(
    max_qubits=50,  # Smaller for composition
    include_qldpc=True,
    include_subsystem=False,  # Skip subsystem for CSS operations
    include_floquet=False,    # Skip floquet for CSS operations
    include_bosonic=False,
    include_qudit=False,
    include_non_css=True,
    timeout_per_code=3.0,
)

# Filter to CSS codes only (required for concatenation and homological product)
css_codes = {}
for name, code in all_codes.items():
    if hasattr(code, 'hx') and hasattr(code, 'hz'):
        # Additional check: must have k > 0
        if code.k > 0:
            css_codes[name] = code

print(f"\nTotal discovered: {len(all_codes)}")
print(f"CSS codes with k>0: {len(css_codes)}")

# Categorize for smart pairing
small_codes = {}   # n <= 10, good for inner codes
medium_codes = {}  # 10 < n <= 30
surface_codes = {} # Surface-like, good for outer codes
other_codes = {}

for name, code in css_codes.items():
    n = code.n
    if 'Surface' in name or 'Toric' in name:
        surface_codes[name] = code
    elif n <= 10:
        small_codes[name] = code
    elif n <= 30:
        medium_codes[name] = code
    else:
        other_codes[name] = code

print(f"\nBy size:")
print(f"  Small (n≤10): {len(small_codes)}")
print(f"  Medium (10<n≤30): {len(medium_codes)}")
print(f"  Surface-like: {len(surface_codes)}")
print(f"  Other: {len(other_codes)}")

# List codes
print("\n" + "-"*70)
print(f"{'Code Name':<40} {'n':>4} {'k':>3} {'d':>3}")
print("-"*70)
for name, code in sorted(css_codes.items(), key=lambda x: x[1].n)[:20]:
    d = code.metadata.get('distance', '?')
    print(f"{name:<40} {code.n:>4} {code.k:>3} {d:>3}")
if len(css_codes) > 20:
    print(f"... and {len(css_codes) - 20} more")

DISCOVERING BUILDING BLOCK CODES

Total discovered: 61
CSS codes with k>0: 50

By size:
  Small (n≤10): 13
  Medium (10<n≤30): 15
  Surface-like: 11
  Other: 11

----------------------------------------------------------------------
Code Name                                   n   k   d
----------------------------------------------------------------------
Repetition_3                                3   1   3
FourQubit422_[[4,2,2]]                      4   2   2
Repetition_5                                5   1   5
C6                                          6   2   2
Steane_713                                  7   1   3
Hamming_CSS_7                               7   1   3
Repetition_7                                7   1   7
TriangularColour_d3                         7   1   3
Code_832                                    8   3   2
LoopToricCode4D                             8   2   ?
HexagonalColour_d2                          8   2   2
Shor_91                                     9   

In [12]:
"""Cell 3: Test Concatenation - All Combinations"""
clear_output(wait=True)

print("="*70)
print("CONCATENATION TESTS")
print("="*70)
print("\nTesting ConcatenatedTopologicalCSSCode(outer, inner)")
print("Expected: n_concat = n_outer × n_inner\n")

concat_results = []

# Generate test pairs: use small codes as inner, surface/medium as outer
# Limit to avoid combinatorial explosion
inner_candidates = list(small_codes.items()) 
outer_candidates = (list(surface_codes.items()) + list(medium_codes.items()))

test_pairs = []
for outer_name, outer_code in outer_candidates:
    for inner_name, inner_code in inner_candidates:
        # Skip if would be too large
        expected_n = outer_code.n * inner_code.n
        if expected_n <= 200:  # Keep manageable
            test_pairs.append((outer_name, outer_code, inner_name, inner_code))

print(f"Testing {len(test_pairs)} concatenation pairs...\n")

for i, (outer_name, outer_code, inner_name, inner_code) in enumerate(test_pairs[:10]):
    print(f"[{i+1}/{len(test_pairs)}] {outer_name} ∘ {inner_name}...", end=' ', flush=True)
    
    result = {
        'outer': outer_name,
        'inner': inner_name,
        'status': 'UNKNOWN',
        'n': None,
        'k': None,
        'circuit_ok': False,
        'error': None,
    }
    
    try:
        # Test construction
        success, concat, msg = test_composite_construction(
            outer_code, inner_code, ConcatenatedTopologicalCSSCode
        )
        
        if success and concat:
            result['n'] = concat.n
            result['k'] = concat.k
            
            # Verify expected n
            expected_n = outer_code.n * inner_code.n
            if concat.n != expected_n:
                result['status'] = 'WARN'
                result['error'] = f"n={concat.n} != expected {expected_n}"
            else:
                # Test circuit generation
                circ_ok, circuit, circ_msg = test_code_circuit(concat)
                result['circuit_ok'] = circ_ok
                
                if circ_ok:
                    result['status'] = 'OK'
                else:
                    result['status'] = 'WARN'
                    result['error'] = circ_msg[:30]
        else:
            result['status'] = 'FAIL'
            result['error'] = msg[:40]
            
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = f"{type(e).__name__}: {str(e)[:30]}"
    
    concat_results.append(result)
    
    # Print status
    if result['status'] == 'OK':
        print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
    elif result['status'] == 'WARN':
        print(f"{STATUS_WARN} {result['error']}")
    else:
        print(f"{STATUS_FAIL} {result['error']}")

# Summary
ok = sum(1 for r in concat_results if r['status'] == 'OK')
warn = sum(1 for r in concat_results if r['status'] == 'WARN')
fail = sum(1 for r in concat_results if r['status'] == 'FAIL')

print("\n" + "-"*70)
print(f"CONCATENATION SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

CONCATENATION TESTS

Testing ConcatenatedTopologicalCSSCode(outer, inner)
Expected: n_concat = n_outer × n_inner

Testing 288 concatenation pairs...

[1/288] RotatedSurface_[[9,1,3]] ∘ FourQubit422_[[4,2,2]]... ✗ ImportError: cannot import name 'SurgeryOpe
[2/288] RotatedSurface_[[9,1,3]] ∘ C6... ✗ ImportError: cannot import name 'SurgeryOpe
[2/288] RotatedSurface_[[9,1,3]] ∘ C6... ✓ n=54, k=10
[3/288] RotatedSurface_[[9,1,3]] ∘ Steane_713... ✓ n=54, k=10
[3/288] RotatedSurface_[[9,1,3]] ∘ Steane_713... ✓ n=63, k=1
[4/288] RotatedSurface_[[9,1,3]] ∘ Shor_91... ✓ n=63, k=1
[4/288] RotatedSurface_[[9,1,3]] ∘ Shor_91... ✓ n=81, k=1
[5/288] RotatedSurface_[[9,1,3]] ∘ Hamming_CSS_7... ✓ n=81, k=1
[5/288] RotatedSurface_[[9,1,3]] ∘ Hamming_CSS_7... ✓ n=63, k=1
[6/288] RotatedSurface_[[9,1,3]] ∘ Code_832... ✗ ValueError: Hx Hz^T != 0 mod 2; not a va
[7/288] RotatedSurface_[[9,1,3]] ∘ Repetition_3... ✓ n=63, k=1
[6/288] RotatedSurface_[[9,1,3]] ∘ Code_832... ✗ ValueError: Hx Hz^T != 0 mod 2; n

In [14]:
"""Cell 4: Test Homological Product"""
clear_output(wait=True)

print("="*70)
print("HOMOLOGICAL PRODUCT TESTS")
print("="*70)
print("\nTesting HomologicalProductCode(code_a, code_b)")
print("Requires codes with chain complexes (topological codes)\n")

# Check which codes have chain complexes - check for attribute directly
codes_with_complex = {}
for name, code in css_codes.items():
    # Check if code has a chain_complex attribute that is not None
    if hasattr(code, 'chain_complex') and code.chain_complex is not None:
        codes_with_complex[name] = code

print(f"CSS codes with chain_complex attribute: {len(codes_with_complex)}")
if codes_with_complex:
    print(f"  Found: {list(codes_with_complex.keys())[:5]}...")

if len(codes_with_complex) < 2:
    # Try importing specific topological codes directly
    print("\nInsufficient codes with chain complexes found in discovery.")
    print("Importing topological codes directly...")
    
    try:
        from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
        from qectostim.codes.surface.toric_code import ToricCode
        
        # Create codes and verify they have chain complexes
        rs3 = RotatedSurfaceCode(distance=3)
        if hasattr(rs3, 'chain_complex') and rs3.chain_complex is not None:
            codes_with_complex['RotatedSurface_d3'] = rs3
            print(f"  ✓ Added RotatedSurface_d3 (n={rs3.n})")
        
        tc3 = ToricCode(Lx=3, Ly=3)
        if hasattr(tc3, 'chain_complex') and tc3.chain_complex is not None:
            codes_with_complex['ToricCode_3x3'] = tc3
            print(f"  ✓ Added ToricCode_3x3 (n={tc3.n})")
            
    except Exception as e:
        print(f"  ✗ Failed: {type(e).__name__}: {e}")
    
    try:
        from qectostim.codes.base.repetition import RepetitionCode
        rep3 = RepetitionCode(N=3)
        if hasattr(rep3, 'chain_complex') and rep3.chain_complex is not None:
            codes_with_complex['Repetition_3'] = rep3
            print(f"  ✓ Added Repetition_3 (n={rep3.n})")
    except Exception as e:
        print(f"  Repetition: {type(e).__name__}: {e}")

hom_results = []

if len(codes_with_complex) >= 1:
    # Generate pairs for homological product - including self-products
    complex_list = list(codes_with_complex.items())
    
    hom_pairs = []
    for i, (name_a, code_a) in enumerate(complex_list):
        for j, (name_b, code_b) in enumerate(complex_list):
            if j >= i:  # Include self-products (A ⊗ A)
                # Check size constraint
                if code_a.n * code_b.n <= 500:
                    hom_pairs.append((name_a, code_a, name_b, code_b))
    
    print(f"\nTesting {len(hom_pairs)} homological product pairs...\n")
    
    for i, (name_a, code_a, name_b, code_b) in enumerate(hom_pairs[10:]):
        print(f"[{i+1}/{len(hom_pairs)}] {name_a} ⊗ {name_b}...", end=' ', flush=True)
        
        result = {
            'code_a': name_a,
            'code_b': name_b,
            'status': 'UNKNOWN',
            'n': None,
            'k': None,
            'error': None,
        }
        
        try:
            # Try direct construction first
            product = HomologicalProductCode(code_a, code_b)
            result['n'] = product.n
            result['k'] = product.k
            
            if product.k > 0:
                result['status'] = 'OK'
            else:
                result['status'] = 'WARN'
                result['error'] = f"k={product.k} (expected k>0)"
                
        except Exception as e:
            result['status'] = 'FAIL'
            result['error'] = f"{type(e).__name__}: {str(e)[:40]}"
        
        hom_results.append(result)
        
        if result['status'] == 'OK':
            print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
        elif result['status'] == 'WARN':
            print(f"{STATUS_WARN} n={result['n']}, {result['error']}")
        else:
            print(f"{STATUS_FAIL} {result['error']}")
else:
    print("\n⚠️ Skipping homological product tests - no codes with chain complexes found.")

# Summary
if hom_results:
    ok = sum(1 for r in hom_results if r['status'] == 'OK')
    warn = sum(1 for r in hom_results if r['status'] == 'WARN')
    fail = sum(1 for r in hom_results if r['status'] == 'FAIL')
    print("\n" + "-"*70)
    print(f"HOMOLOGICAL PRODUCT SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

HOMOLOGICAL PRODUCT TESTS

Testing HomologicalProductCode(code_a, code_b)
Requires codes with chain complexes (topological codes)

CSS codes with chain_complex attribute: 25
  Found: ['FourQubit422_[[4,2,2]]', 'Steane_713', 'Shor_91', 'Hamming_CSS_7', 'Hamming_CSS_15']...

Testing 202 homological product pairs...

[1/202] FourQubit422_[[4,2,2]] ⊗ FourQubit422_[[4,2,2]]... ✓ n=18, k=4
[2/202] FourQubit422_[[4,2,2]] ⊗ Steane_713... ✓ n=18, k=4
[2/202] FourQubit422_[[4,2,2]] ⊗ Steane_713... ✓ n=34, k=2
[3/202] FourQubit422_[[4,2,2]] ⊗ Shor_91... ✓ n=34, k=2
[3/202] FourQubit422_[[4,2,2]] ⊗ Shor_91... ✓ n=44, k=2
[4/202] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_7... ✓ n=44, k=2
[4/202] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_7... ✓ n=34, k=2
[5/202] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_15... ✓ n=34, k=2
[5/202] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_15... ✓ n=68, k=14
[6/202] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_31... ✓ n=68, k=14
[6/202] FourQubit422_[[4,2,2]] ⊗ Hamming_CSS_31... ✓ n=134, k=42
[7/

KeyboardInterrupt: 

In [15]:
"""Cell 5: Multi-Level Concatenation"""
clear_output(wait=True)

print("="*70)
print("MULTI-LEVEL CONCATENATION TESTS")
print("="*70)
print("\nTesting nested concatenation: (outer ∘ middle) ∘ inner")
print("Demonstrates composability of composite operations\n")

multi_results = []

# Select small codes for multi-level (to keep size manageable)
small_list = list(small_codes.items())[:4]

# Test cases for 2-level concatenation
multilevel_cases = []

# Pattern: (A ∘ B) ∘ C
if len(small_list) >= 3:
    for i in range(min(3, len(small_list))):
        outer = small_list[i]
        middle = small_list[(i+1) % len(small_list)]
        inner = small_list[(i+2) % len(small_list)]
        
        # Check size
        expected_n = outer[1].n * middle[1].n * inner[1].n
        if expected_n <= 500:
            multilevel_cases.append((outer, middle, inner))

print(f"Testing {len(multilevel_cases)} multi-level concatenations...\n")

for i, (outer, middle, inner) in enumerate(multilevel_cases[:10]):
    outer_name, outer_code = outer
    middle_name, middle_code = middle
    inner_name, inner_code = inner
    
    desc = f"({outer_name} ∘ {middle_name}) ∘ {inner_name}"
    print(f"[{i+1}/{len(multilevel_cases)}] {desc}...", end=' ', flush=True)
    
    result = {
        'desc': desc,
        'status': 'UNKNOWN',
        'n': None,
        'k': None,
        'error': None,
    }
    
    try:
        # Level 1: outer ∘ middle
        level1 = ConcatenatedTopologicalCSSCode(outer_code, middle_code)
        
        # Level 2: (outer ∘ middle) ∘ inner
        level2 = ConcatenatedTopologicalCSSCode(level1, inner_code)
        
        result['n'] = level2.n
        result['k'] = level2.k
        
        # Verify expected n
        expected_n = outer_code.n * middle_code.n * inner_code.n
        if level2.n == expected_n and level2.k > 0:
            result['status'] = 'OK'
        else:
            result['status'] = 'WARN'
            result['error'] = f"n={level2.n} (expected {expected_n}), k={level2.k}"
            
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = f"{type(e).__name__}: {str(e)[:30]}"
    
    multi_results.append(result)
    
    if result['status'] == 'OK':
        print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
    elif result['status'] == 'WARN':
        print(f"{STATUS_WARN} {result['error']}")
    else:
        print(f"{STATUS_FAIL} {result['error']}")

# Summary
if multi_results:
    ok = sum(1 for r in multi_results if r['status'] == 'OK')
    warn = sum(1 for r in multi_results if r['status'] == 'WARN')
    fail = sum(1 for r in multi_results if r['status'] == 'FAIL')
    print("\n" + "-"*70)
    print(f"MULTI-LEVEL SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

MULTI-LEVEL CONCATENATION TESTS

Testing nested concatenation: (outer ∘ middle) ∘ inner
Demonstrates composability of composite operations

Testing 3 multi-level concatenations...

[1/3] (FourQubit422_[[4,2,2]] ∘ C6) ∘ Steane_713... ✓ n=168, k=6
[2/3] (C6 ∘ Steane_713) ∘ Shor_91... ✓ n=168, k=6
[2/3] (C6 ∘ Steane_713) ∘ Shor_91... 

✓ n=378, k=2
[3/3] (Steane_713 ∘ Shor_91) ∘ FourQubit422_[[4,2,2]]... ✓ n=252, k=64

----------------------------------------------------------------------
MULTI-LEVEL SUMMARY: 3 ✓ | 0 ⚠️ | 0 ✗
✓ n=252, k=64

----------------------------------------------------------------------
MULTI-LEVEL SUMMARY: 3 ✓ | 0 ⚠️ | 0 ✗


In [16]:
# Initialize hom_results and dual_results if not defined
if 'hom_results' not in dir():
    hom_results = []
if 'dual_results' not in dir():
    dual_results = []

# Also initialize codes_with_complex if needed
if 'codes_with_complex' not in dir():
    codes_with_complex = {}

In [ ]:
"""Cell 6: Decoder Smoke Test on ALL Composite Codes - ALL DECODERS"""
import sys
from IPython.display import clear_output
clear_output(wait=True)

# Clear qectostim modules to pick up any code fixes
modules_to_clear = [k for k in list(sys.modules.keys()) if 'qectostim' in k]
for mod in modules_to_clear:
    del sys.modules[mod]

# Re-import essential modules after clearing
from qectostim.codes.discovery import discover_all_codes
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.testing import test_decoder_on_code, load_all_decoders

# Reload decoder classes and code lists
decoder_classes = load_all_decoders()
all_codes = discover_all_codes()
css_codes = {n: c for n, c in all_codes.items() 
             if hasattr(c, 'hx') and hasattr(c, 'hz') and c.k > 0}

print("="*100)
print("DECODER SMOKE TEST ON COMPOSITE CODES - ALL DECODERS")
print("="*100)

if len(decoder_classes) == 0:
    print("\n⚠️ No decoders available - skipping decoder tests")
else:
    # Use ALL decoders - we've fixed the issues!
    # Chromobius will SKIP for non-color codes (expected)
    # MLE will SKIP for large codes (expected)
    test_decoders = decoder_classes
    
    print(f"\nTesting {len(test_decoders)} decoders on composite codes")
    print(f"Decoders: {list(test_decoders.keys())}")
    print("\nNote: Chromobius skips non-color codes, MLE skips large codes (expected)")
    
    all_decoder_results = []
    
    # Helper function to test one composite code with all decoders
    def test_composite_with_decoders(code, code_label, code_type='CSS'):
        """Test a composite code with all decoders and return results row."""
        row_results = {'code': code_label, 'results': {}}
        row = f"  {code_label:<35}"
        
        for dec_name, dec_class in test_decoders.items():
            try:
                test_res = test_decoder_on_code(
                    code,
                    dec_class,
                    decoder_name=dec_name,
                    code_type=code_type,
                    p=0.01,
                    shots=500,
                    rounds=2,
                )
                
                if test_res.status == 'OK':
                    ler_str = f"✓{test_res.ler:.3f}" if test_res.ler is not None else "✓---"
                elif test_res.status == 'SKIP':
                    ler_str = "-skip"
                elif test_res.status == 'WARN':
                    ler_str = f"⚠{test_res.ler:.3f}" if test_res.ler is not None else "⚠---"
                else:
                    ler_str = "✗fail"
                    
                row += f" {ler_str:>8}"
                row_results['results'][dec_name] = {
                    'status': test_res.status,
                    'ler': test_res.ler,
                    'error': test_res.error if test_res.status not in ['OK', 'SKIP'] else None
                }
                
            except Exception as e:
                row += f" {'✗err':>8}"
                row_results['results'][dec_name] = {
                    'status': 'FAIL',
                    'error': str(e)[:50]
                }
        
        print(row)
        return row_results
    
    # Print header
    header = f"  {'Composite Code':<35}"
    for dec_name in test_decoders:
        header += f" {dec_name[:8]:>8}"
    print(header)
    print("-" * len(header))
    
    # =========================================================================
    # 1. CONCATENATED CODES
    # =========================================================================
    print("\n[CONCATENATED CODES]")
    successful_concats = [r for r in concat_results if r['status'] == 'OK'][:5]  # Limit to 5
    
    for concat_res in successful_concats:
        outer_name = concat_res['outer']
        inner_name = concat_res['inner']
        
        try:
            outer_code = css_codes.get(outer_name)
            inner_code = css_codes.get(inner_name)
            
            if outer_code is None or inner_code is None:
                print(f"  {outer_name[:15]}∘{inner_name[:15]:<19} Code not found")
                continue
                
            concat = ConcatenatedTopologicalCSSCode(outer_code, inner_code)
            code_label = f"{outer_name[:15]}∘{inner_name[:15]}"
            res = test_composite_with_decoders(concat, code_label)
            all_decoder_results.append(res)
            
        except Exception as e:
            print(f"  {outer_name[:15]}∘{inner_name[:15]:<19} Build error: {type(e).__name__}")
    
    # =========================================================================
    # 2. MULTI-LEVEL CONCATENATED CODES
    # =========================================================================
    if multi_results:
        print("\n[MULTI-LEVEL CONCATENATED]")
        successful_multi = [r for r in multi_results if r['status'] == 'OK'][:3]  # Limit to 3
        small_list = list(small_codes.items())
        
        for i, multi_res in enumerate(successful_multi):
            try:
                if len(small_list) >= 3:
                    outer = small_list[i][1]
                    middle = small_list[(i+1) % len(small_list)][1]
                    inner = small_list[(i+2) % len(small_list)][1]
                    
                    level1 = ConcatenatedTopologicalCSSCode(outer, middle)
                    level2 = ConcatenatedTopologicalCSSCode(level1, inner)
                    
                    code_label = f"({small_list[i][0][:6]}∘{small_list[(i+1)%len(small_list)][0][:6]})∘{small_list[(i+2)%len(small_list)][0][:5]}"
                    res = test_composite_with_decoders(level2, code_label)
                    all_decoder_results.append(res)
            except Exception as e:
                print(f"  Multi-level {i+1}: Build error: {type(e).__name__}: {str(e)[:30]}")
    
    # =========================================================================
    # 3. HOMOLOGICAL PRODUCT CODES
    # =========================================================================
    if hom_results:
        print("\n[HOMOLOGICAL PRODUCT]")
        successful_hom = [r for r in hom_results if r['status'] == 'OK'][:3]  # Limit to 3
        
        for hom_res in successful_hom:
            code_a_name = hom_res['code_a']
            code_b_name = hom_res['code_b']
            
            try:
                code_a = codes_with_complex.get(code_a_name)
                code_b = codes_with_complex.get(code_b_name)
                
                if code_a is None or code_b is None:
                    print(f"  {code_a_name[:15]}⊗{code_b_name[:15]:<19} Code not found")
                    continue
                    
                product = HomologicalProductCode(code_a, code_b)
                code_label = f"{code_a_name[:15]}⊗{code_b_name[:15]}"
                res = test_composite_with_decoders(product, code_label)
                all_decoder_results.append(res)
                
            except Exception as e:
                print(f"  {code_a_name[:15]}⊗{code_b_name[:15]:<19} Build error: {type(e).__name__}")
    
    # =========================================================================
    # 4. DUAL CODES (only if defined - run Cell 7 first if needed)
    # =========================================================================
    if 'dual_results' in dir() or 'dual_results' in globals():
        print("\n[DUAL CODES]")
        successful_dual = [r for r in dual_results if r['status'] == 'OK'][:3]
        
        for dual_res in successful_dual:
            orig_name = dual_res['original']
            
            try:
                orig_code = css_codes.get(orig_name)
                if orig_code is None:
                    print(f"  Dual({orig_name[:25]:<25}) Code not found")
                    continue
                    
                dual = DualCode(orig_code)
                code_label = f"Dual({orig_name[:25]})"
                res = test_composite_with_decoders(dual, code_label)
                all_decoder_results.append(res)
                
            except Exception as e:
                print(f"  Dual({orig_name[:25]:<25}) Build error: {type(e).__name__}")
    else:
        print("\n[DUAL CODES] Skipped - run Cell 7 first to define dual_results")
    
    print("-" * len(header))
    
    # Summary
    if all_decoder_results:
        ok_count = sum(1 for r in all_decoder_results 
                       for d in r['results'].values() 
                       if d.get('status') == 'OK')
        skip_count = sum(1 for r in all_decoder_results 
                         for d in r['results'].values() 
                         if d.get('status') == 'SKIP')
        fail_count = sum(1 for r in all_decoder_results 
                         for d in r['results'].values() 
                         if d.get('status') == 'FAIL')
        warn_count = sum(1 for r in all_decoder_results 
                         for d in r['results'].values() 
                         if d.get('status') == 'WARN')
        
        total = ok_count + skip_count + fail_count + warn_count
        print(f"\nDECODER SUMMARY:")
        print(f"  Total tests: {total}")
        print(f"  {STATUS_OK} Pass: {ok_count} ({100*ok_count/total if total else 0:.0f}%)")
        print(f"  {STATUS_WARN} Warn: {warn_count}")
        print(f"  {STATUS_SKIP} Skip: {skip_count} (expected for Chromobius/MLE on large codes)")
        print(f"  {STATUS_FAIL} Fail: {fail_count}")
        
        # Show failures with details
        failures = [(r['code'], d_name, d_res.get('error'))
                    for r in all_decoder_results
                    for d_name, d_res in r['results'].items()
                    if d_res.get('status') == 'FAIL' and d_res.get('error')]
        
        if failures:
            print(f"\n{STATUS_FAIL} Failure details ({len(failures)}):")
            for code, dec, err in failures[:15]:
                print(f"  {code} + {dec}: {err}")

DECODER SMOKE TEST ON COMPOSITE CODES - ALL DECODERS

Testing 11 decoders on composite codes
Decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat']

Note: Chromobius skips non-color codes, MLE skips large codes (expected)
  Composite Code                      PyMatchi FusionBl BeliefMa    BPOSD Tesserac UnionFin      MLE Hypergra Chromobi Concaten FlatConc
----------------------------------------------------------------------------------------------------------------------------------------

[CONCATENATED CODES]
  RotatedSurface_∘C6                      ✗err   ✓0.214   ✓0.344   ✓0.332   ✓0.154   ✓0.144    -skip   ✓0.154    -skip    ✗fail   ✓0.152
  RotatedSurface_∘C6                      ✗err   ✓0.214   ✓0.344   ✓0.332   ✓0.154   ✓0.144    -skip   ✓0.154    -skip    ✗fail   ✓0.152


In [ ]:
"""Cell 6b: Decoder Results Summary"""
# Summarize decoder results from Cell 6
print("="*80)
print("DECODER RESULTS SUMMARY")
print("="*80)

if 'all_decoder_results' in dir() and all_decoder_results:
    # Count by decoder
    decoder_stats = {}
    for res in all_decoder_results:
        for dec_name, dec_res in res['results'].items():
            if dec_name not in decoder_stats:
                decoder_stats[dec_name] = {'OK': 0, 'SKIP': 0, 'FAIL': 0, 'WARN': 0}
            status = dec_res.get('status', 'FAIL')
            decoder_stats[dec_name][status] = decoder_stats[dec_name].get(status, 0) + 1
    
    print(f"\n{'Decoder':<15} {'OK':>6} {'SKIP':>6} {'WARN':>6} {'FAIL':>6}")
    print("-"*45)
    for dec_name in sorted(decoder_stats.keys()):
        stats = decoder_stats[dec_name]
        print(f"{dec_name:<15} {stats['OK']:>6} {stats['SKIP']:>6} {stats['WARN']:>6} {stats['FAIL']:>6}")
    
    # Show all failures with full error messages
    print("\n" + "="*80)
    print("FAILURE DETAILS")
    print("="*80)
    
    failures = []
    for res in all_decoder_results:
        for dec_name, dec_res in res['results'].items():
            if dec_res.get('status') == 'FAIL':
                failures.append({
                    'code': res['code'],
                    'decoder': dec_name,
                    'error': dec_res.get('error', 'Unknown error')
                })
    
    if failures:
        # Group failures by decoder
        from collections import defaultdict
        by_decoder = defaultdict(list)
        for f in failures:
            by_decoder[f['decoder']].append(f)
        
        for dec_name, dec_failures in sorted(by_decoder.items()):
            print(f"\n{dec_name} ({len(dec_failures)} failures):")
            # Show first 3 unique errors
            seen_errors = set()
            for f in dec_failures[:5]:
                err = f['error'] or 'Unknown'
                if err not in seen_errors:
                    seen_errors.add(err)
                    print(f"  {f['code']}: {err[:70]}")
    else:
        print("\n✓ No failures!")
else:
    print("\n⚠️ No decoder results found - run Cell 6 first")

DECODER RESULTS SUMMARY

Decoder             OK   SKIP   WARN   FAIL
---------------------------------------------
BPOSD               14      0      0      0
BeliefMatching      14      0      0      0
Chromobius           0     14      0      0
Concatenated         0      0      8      6
FusionBlossom       14      0      0      0
Hypergraph           8      0      6      0
MLE                  3     11      0      0
PyMatching           8      0      6      0
Tesseract           14      0      0      0
UnionFind            8      0      6      0

FAILURE DETAILS

Concatenated (6 failures):
  FourQubit422_[[⊗FourQubit422_[[: Decoder init: ConcatenatedDecoder requi


In [ ]:
"""Cell 7: Dual Code Tests"""
clear_output(wait=True)

print("="*70)
print("DUAL CODE TESTS")
print("="*70)
print("\nTesting DualCode(code) - swaps X and Z sectors\n")

dual_results = []

# Test on a selection of codes
dual_test_codes = list(css_codes.items())[:10]

for i, (name, code) in enumerate(dual_test_codes):
    print(f"[{i+1}/{len(dual_test_codes)}] Dual({name})...", end=' ', flush=True)
    
    result = {
        'original': name,
        'status': 'UNKNOWN',
        'n': None,
        'k': None,
        'error': None,
    }
    
    try:
        dual = DualCode(code)
        result['n'] = dual.n
        result['k'] = dual.k
        
        # Verify n and k preserved
        if dual.n == code.n and dual.k == code.k:
            result['status'] = 'OK'
        else:
            result['status'] = 'WARN'
            result['error'] = f"n,k changed: {code.n},{code.k} -> {dual.n},{dual.k}"
            
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = f"{type(e).__name__}: {str(e)[:30]}"
    
    dual_results.append(result)
    
    if result['status'] == 'OK':
        print(f"{STATUS_OK} n={result['n']}, k={result['k']}")
    elif result['status'] == 'WARN':
        print(f"{STATUS_WARN} {result['error']}")
    else:
        print(f"{STATUS_FAIL} {result['error']}")

# Summary
ok = sum(1 for r in dual_results if r['status'] == 'OK')
warn = sum(1 for r in dual_results if r['status'] == 'WARN')
fail = sum(1 for r in dual_results if r['status'] == 'FAIL')

print("\n" + "-"*70)
print(f"DUAL CODE SUMMARY: {ok} {STATUS_OK} | {warn} {STATUS_WARN} | {fail} {STATUS_FAIL}")

DUAL CODE TESTS

Testing DualCode(code) - swaps X and Z sectors

[1/10] Dual(FourQubit422_[[4,2,2]])... 

✓ n=4, k=2
[2/10] Dual(C6)... ✓ n=6, k=2
[3/10] Dual(Steane_713)... ✓ n=6, k=2
[3/10] Dual(Steane_713)... ✓ n=7, k=1
[4/10] Dual(Shor_91)... ✓ n=9, k=1
[5/10] Dual(ReedMuller_15_1_3)... ✓ n=7, k=1
[4/10] Dual(Shor_91)... ✓ n=9, k=1
[5/10] Dual(ReedMuller_15_1_3)... ✓ n=15, k=1
[6/10] Dual(Hamming_CSS_7)... ✓ n=7, k=1
[7/10] Dual(Hamming_CSS_15)... ✓ n=15, k=1
[6/10] Dual(Hamming_CSS_7)... ✓ n=7, k=1
[7/10] Dual(Hamming_CSS_15)... ✓ n=15, k=7
[8/10] Dual(Hamming_CSS_31)... ✓ n=31, k=21
[9/10] Dual(Code_832)... ✓ n=8, k=3
[10/10] Dual(Repetition_3)... ✓ n=15, k=7
[8/10] Dual(Hamming_CSS_31)... ✓ n=31, k=21
[9/10] Dual(Code_832)... ✓ n=8, k=3
[10/10] Dual(Repetition_3)... ✓ n=3, k=1

----------------------------------------------------------------------
DUAL CODE SUMMARY: 10 ✓ | 0 ⚠️ | 0 ✗
✓ n=3, k=1

----------------------------------------------------------------------
DUAL CODE SUMMARY: 10 ✓ | 0 ⚠️ | 0 ✗


In [ ]:
"""Cell 8: Comprehensive Results Table"""
clear_output(wait=True)

print("="*100)
print("COMPOSITE CODE SMOKE TEST - COMPREHENSIVE RESULTS")
print("="*100)

# Concatenation results
print("\nCONCATENATION (ConcatenatedTopologicalCSSCode):")
print("-"*100)
print(f"{'Outer':<20} | {'Inner':<20} | {'n':>5} | {'k':>3} | {'Circuit':>8} | Status")
print("-"*100)

for r in concat_results:
    outer = r['outer'][:20] if r['outer'] else 'N/A'
    inner = r['inner'][:20] if r['inner'] else 'N/A'
    n = str(r['n']) if r['n'] else '-'
    k = str(r['k']) if r['k'] else '-'
    circ = STATUS_OK if r.get('circuit_ok') else STATUS_FAIL
    
    if r['status'] == 'OK':
        status = f"{STATUS_OK} OK"
    elif r['status'] == 'WARN':
        status = f"{STATUS_WARN} {r['error'][:20]}" if r['error'] else f"{STATUS_WARN}"
    else:
        status = f"{STATUS_FAIL} {r['error'][:20]}" if r['error'] else f"{STATUS_FAIL}"
    
    print(f"{outer:<20} | {inner:<20} | {n:>5} | {k:>3} | {circ:>8} | {status}")

# Multi-level results
if multi_results:
    print("\n\nMULTI-LEVEL CONCATENATION:")
    print("-"*100)
    for r in multi_results:
        desc = r['desc'][:50] if r.get('desc') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK} OK"
        elif r['status'] == 'WARN':
            status = f"{STATUS_WARN}"
        else:
            status = f"{STATUS_FAIL} {r['error'][:30]}" if r['error'] else f"{STATUS_FAIL}"
        
        print(f"{desc:<50} | n={n:>5} k={k:>3} | {status}")

# Homological product results
if hom_results:
    print("\n\nHOMOLOGICAL PRODUCT:")
    print("-"*100)
    for r in hom_results:
        code_a = r['code_a'][:20] if r.get('code_a') else 'N/A'
        code_b = r['code_b'][:20] if r.get('code_b') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK} OK"
        else:
            status = f"{STATUS_FAIL} {r['error'][:30]}" if r['error'] else f"{STATUS_FAIL}"
        
        print(f"{code_a:<20} ⊗ {code_b:<20} | n={n:>5} k={k:>3} | {status}")

# Dual code results
if dual_results:
    print("\n\nDUAL CODES:")
    print("-"*100)
    for r in dual_results:
        orig = r['original'][:30] if r.get('original') else 'N/A'
        n = str(r['n']) if r['n'] else '-'
        k = str(r['k']) if r['k'] else '-'
        
        if r['status'] == 'OK':
            status = f"{STATUS_OK}"
        elif r['status'] == 'WARN':
            status = f"{STATUS_WARN}"
        else:
            status = f"{STATUS_FAIL}"
        
        print(f"Dual({orig:<30}) | n={n:>5} k={k:>3} | {status}")

COMPOSITE CODE SMOKE TEST - COMPREHENSIVE RESULTS

CONCATENATION (ConcatenatedTopologicalCSSCode):
----------------------------------------------------------------------------------------------------
Outer                | Inner                |     n |   k |  Circuit | Status
----------------------------------------------------------------------------------------------------
RotatedSurface_[[9,1 | FourQubit422_[[4,2,2 |    36 |  10 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | C6                   |    54 |  10 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Steane_713           |    63 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Shor_91              |    81 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Hamming_CSS_7        |    63 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Code_832             |     - |   - |        ✗ | ✗ ValueError: Hx Hz^T 
RotatedSurface_[[9,1 | Repetition_3         |    27 |   1 |        ✓ | ✓ OK
RotatedSurface_[[9,1 | Repetition_5         |    45 |   1 |        ✓ | 

In [ ]:
"""Cell 9: Final Summary"""
print("\n" + "="*70)
print("COMPOSITE CODE SMOKE TEST - FINAL SUMMARY")
print("="*70)

# Count totals
concat_ok = sum(1 for r in concat_results if r['status'] == 'OK')
concat_total = len(concat_results)

multi_ok = sum(1 for r in multi_results if r['status'] == 'OK') if multi_results else 0
multi_total = len(multi_results) if multi_results else 0

hom_ok = sum(1 for r in hom_results if r['status'] == 'OK') if hom_results else 0
hom_total = len(hom_results) if hom_results else 0

dual_ok = sum(1 for r in dual_results if r['status'] == 'OK') if dual_results else 0
dual_total = len(dual_results) if dual_results else 0

print(f"\n{'Operation':<30} | {'Passed':>8} | {'Total':>8} | {'Rate':>8}")
print("-"*60)
print(f"{'Concatenation':<30} | {concat_ok:>8} | {concat_total:>8} | {100*concat_ok/concat_total if concat_total else 0:.0f}%")
print(f"{'Multi-level Concatenation':<30} | {multi_ok:>8} | {multi_total:>8} | {100*multi_ok/multi_total if multi_total else 0:.0f}%")
print(f"{'Homological Product':<30} | {hom_ok:>8} | {hom_total:>8} | {100*hom_ok/hom_total if hom_total else 0:.0f}%")
print(f"{'Dual Code':<30} | {dual_ok:>8} | {dual_total:>8} | {100*dual_ok/dual_total if dual_total else 0:.0f}%")
print("-"*60)

total_ok = concat_ok + multi_ok + hom_ok + dual_ok
total_tests = concat_total + multi_total + hom_total + dual_total

print(f"{'TOTAL':<30} | {total_ok:>8} | {total_tests:>8} | {100*total_ok/total_tests if total_tests else 0:.0f}%")

print("\n" + "="*70)
print(f"Building block codes used: {len(css_codes)}")
print(f"Composite constructions tested: {total_tests}")

if total_ok == total_tests:
    print(f"\n{STATUS_OK} ALL COMPOSITE TESTS PASSED")
else:
    print(f"\n{STATUS_WARN} {total_tests - total_ok} tests need attention")
print("="*70)


COMPOSITE CODE SMOKE TEST - FINAL SUMMARY

Operation                      |   Passed |    Total |     Rate
------------------------------------------------------------
Concatenation                  |      153 |      252 | 61%
Multi-level Concatenation      |        3 |        3 | 100%
Homological Product            |      122 |      173 | 71%
Dual Code                      |       10 |       10 | 100%
------------------------------------------------------------
TOTAL                          |      288 |      438 | 66%

Building block codes used: 46
Composite constructions tested: 438

⚠️ 150 tests need attention
